In [10]:
import argparse

import torch
import mmcv
from mmcv.runner import load_checkpoint, parallel_test, obj_from_dict
from mmcv.parallel import scatter, collate, MMDataParallel

from mmdet import datasets
from mmdet.core import results2json, coco_eval
from mmdet.datasets import build_dataloader, CocoDataset, CustomDataset
from mmdet.models import build_detector, detectors
import numpy as np
from PIL import Image

def single_test(model, data_loader, gt, show=False):
    model.eval()
    results = []
    file_names = []
    dataset = data_loader.dataset
    prog_bar = mmcv.ProgressBar(len(dataset))
    for i, data in enumerate(data_loader):
        print (i)
        with torch.no_grad():
            result = model(return_loss=False, rescale=not show, **data)
        results.append(result)
        file_names.append(gt['images'][i]["file_name"])
        if show:
            model.module.show_result(data, result, dataset.img_norm_cfg,
                                     dataset=dataset.CLASSES)
        
        batch_size = data['img'][0].size(0)
        for _ in range(batch_size):
            prog_bar.update()

    return results,file_names


In [ ]:
out_file = '/home/hywel/Documents/mmdetection/result.pkl'
image_dir = '/home/hywel/Documents/mmdetection/data/coco/test_b/'
config_file = '/home/hywel/Documents/mmdetection/configs/dcn/cascade_rcnn_dconv_c3-c5_r101_fpn_1x.py'
checkpoint_file = '/home/hywel/Documents/mmdetection/work_dirs/cascade_rcnn_dconv_c3-c5_r101_fpn_1x/latest.pth'
import json
import random
from skimage.io import imsave

datasets_path = '/home/hywel/Documents/mmdetection/data/coco/annotations/'
f = open('{}test_b.json'.format(datasets_path) ,encoding='utf-8')
gt = json.load(f)

cfg = mmcv.Config.fromfile(config_file)
# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
cfg.model.pretrained = None
cfg.data.test.test_mode = True

dataset = obj_from_dict(cfg.data.test, datasets, dict(test_mode=True))

model = build_detector(
    cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
load_checkpoint(model, checkpoint_file)
model = MMDataParallel(model, device_ids=[0])

data_loader = build_dataloader(
    dataset,
    imgs_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    num_gpus=1,
    dist=False,
    shuffle=False)
outputs,file_names = single_test(model, data_loader, gt,False)


    

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[                              ] 0/3398, elapsed: 0s, ETA:0
[                              ] 1/3398, 0.4 task/s, elapsed: 3s, ETA:  8842s1
[                              ] 2/3398, 0.4 task/s, elapsed: 5s, ETA:  7738s2
[                              ] 3/3398, 0.5 task/s, elapsed: 7s, ETA:  7508s3
[                              ] 4/3398, 0.4 task/s, elapsed: 9s, ETA:  7646s4
[                              ] 5/3398, 0.4 task/s, elapsed: 11s, ETA:  7603s5
[                              ] 6/3398, 0.5 task/s, elapsed: 13s, ETA:  7373s6
[                              ] 7/3398, 0.5 task/s, elapsed: 15s, ETA:  7222s7
[                              ] 8/3398, 0.5 task/s, elapsed: 17s, ETA:  7328s8
[                              ] 9/3398, 0.5 task/s, elapsed: 19s, ETA:  7214s9
[                              ] 10/3398, 0.5 task/s, elapsed: 22s, ETA:  7320s10
[                              ] 11/3398, 0.5 task/s, elaps

[                              ] 98/3398, 0.5 task/s, elapsed: 212s, ETA:  7142s98
[                              ] 99/3398, 0.5 task/s, elapsed: 215s, ETA:  7149s99
[                              ] 100/3398, 0.5 task/s, elapsed: 217s, ETA:  7144s100
[                              ] 101/3398, 0.5 task/s, elapsed: 219s, ETA:  7150s101
[                              ] 102/3398, 0.5 task/s, elapsed: 221s, ETA:  7153s102
[                              ] 103/3398, 0.5 task/s, elapsed: 224s, ETA:  7153s103
[                              ] 104/3398, 0.5 task/s, elapsed: 226s, ETA:  7144s104
[                              ] 105/3398, 0.5 task/s, elapsed: 228s, ETA:  7140s105
[                              ] 106/3398, 0.5 task/s, elapsed: 230s, ETA:  7133s106
[                              ] 107/3398, 0.5 task/s, elapsed: 232s, ETA:  7131s107
[                              ] 108/3398, 0.5 task/s, elapsed: 234s, ETA:  7136s108
[                              ] 109/3398, 0.5 task/s, elapsed: 236s,

[>                             ] 194/3398, 0.5 task/s, elapsed: 417s, ETA:  6893s194
[>                             ] 195/3398, 0.5 task/s, elapsed: 420s, ETA:  6891s195
[>                             ] 196/3398, 0.5 task/s, elapsed: 422s, ETA:  6890s196
[>                             ] 197/3398, 0.5 task/s, elapsed: 423s, ETA:  6876s197
[>                             ] 198/3398, 0.5 task/s, elapsed: 425s, ETA:  6873s198
[>                             ] 199/3398, 0.5 task/s, elapsed: 428s, ETA:  6872s199
[>                             ] 200/3398, 0.5 task/s, elapsed: 429s, ETA:  6867s200
[>                             ] 201/3398, 0.5 task/s, elapsed: 432s, ETA:  6869s201
[>                             ] 202/3398, 0.5 task/s, elapsed: 434s, ETA:  6865s202
[>                             ] 203/3398, 0.5 task/s, elapsed: 436s, ETA:  6861s203
[>                             ] 204/3398, 0.5 task/s, elapsed: 438s, ETA:  6862s204
[>                             ] 205/3398, 0.5 task/s, elapsed: 4

[>>                            ] 290/3398, 0.5 task/s, elapsed: 624s, ETA:  6686s290
[>>                            ] 291/3398, 0.5 task/s, elapsed: 626s, ETA:  6686s291
[>>                            ] 292/3398, 0.5 task/s, elapsed: 628s, ETA:  6682s292
[>>                            ] 293/3398, 0.5 task/s, elapsed: 631s, ETA:  6683s293
[>>                            ] 294/3398, 0.5 task/s, elapsed: 633s, ETA:  6678s294
[>>                            ] 295/3398, 0.5 task/s, elapsed: 635s, ETA:  6680s295
[>>                            ] 296/3398, 0.5 task/s, elapsed: 638s, ETA:  6681s296
[>>                            ] 297/3398, 0.5 task/s, elapsed: 640s, ETA:  6680s297
[>>                            ] 298/3398, 0.5 task/s, elapsed: 642s, ETA:  6678s298
[>>                            ] 299/3398, 0.5 task/s, elapsed: 644s, ETA:  6679s299
[>>                            ] 300/3398, 0.5 task/s, elapsed: 646s, ETA:  6674s300
[>>                            ] 301/3398, 0.5 task/s, elapsed: 6

[>>>                           ] 386/3398, 0.5 task/s, elapsed: 832s, ETA:  6490s386
[>>>                           ] 387/3398, 0.5 task/s, elapsed: 834s, ETA:  6490s387
[>>>                           ] 388/3398, 0.5 task/s, elapsed: 836s, ETA:  6485s388
[>>>                           ] 389/3398, 0.5 task/s, elapsed: 838s, ETA:  6484s389
[>>>                           ] 390/3398, 0.5 task/s, elapsed: 840s, ETA:  6482s390
[>>>                           ] 391/3398, 0.5 task/s, elapsed: 842s, ETA:  6479s391
[>>>                           ] 392/3398, 0.5 task/s, elapsed: 845s, ETA:  6477s392
[>>>                           ] 393/3398, 0.5 task/s, elapsed: 847s, ETA:  6473s393
[>>>                           ] 394/3398, 0.5 task/s, elapsed: 849s, ETA:  6473s394
[>>>                           ] 395/3398, 0.5 task/s, elapsed: 851s, ETA:  6470s395
[>>>                           ] 396/3398, 0.5 task/s, elapsed: 854s, ETA:  6471s396
[>>>                           ] 397/3398, 0.5 task/s, elapsed: 8

...

[>>>>>>>>>>>>>>>>>>>           ] 2260/3398, 0.5 task/s, elapsed: 4860s, ETA:  2447s2260
[>>>>>>>>>>>>>>>>>>>           ] 2261/3398, 0.5 task/s, elapsed: 4862s, ETA:  2445s2261
[>>>>>>>>>>>>>>>>>>>           ] 2262/3398, 0.5 task/s, elapsed: 4864s, ETA:  2443s2262
[>>>>>>>>>>>>>>>>>>>           ] 2263/3398, 0.5 task/s, elapsed: 4866s, ETA:  2441s2263
[>>>>>>>>>>>>>>>>>>>           ] 2264/3398, 0.5 task/s, elapsed: 4868s, ETA:  2438s2264
[>>>>>>>>>>>>>>>>>>>           ] 2265/3398, 0.5 task/s, elapsed: 4870s, ETA:  2436s2265
[>>>>>>>>>>>>>>>>>>>>          ] 2266/3398, 0.5 task/s, elapsed: 4873s, ETA:  2434s2266
[>>>>>>>>>>>>>>>>>>>>          ] 2267/3398, 0.5 task/s, elapsed: 4875s, ETA:  2432s2267
[>>>>>>>>>>>>>>>>>>>>          ] 2268/3398, 0.5 task/s, elapsed: 4877s, ETA:  2430s2268
[>>>>>>>>>>>>>>>>>>>>          ] 2269/3398, 0.5 task/s, elapsed: 4879s, ETA:  2428s2269
[>>>>>>>>>>>>>>>>>>>>          ] 2270/3398, 0.5 task/s, elapsed: 4882s, ETA:  2426s2270
[>>>>>>>>>>>>>>>>>>>>          ]

[>>>>>>>>>>>>>>>>>>>>          ] 2353/3398, 0.5 task/s, elapsed: 5060s, ETA:  2247s2353
[>>>>>>>>>>>>>>>>>>>>          ] 2354/3398, 0.5 task/s, elapsed: 5062s, ETA:  2245s2354
[>>>>>>>>>>>>>>>>>>>>          ] 2355/3398, 0.5 task/s, elapsed: 5064s, ETA:  2243s2355
[>>>>>>>>>>>>>>>>>>>>          ] 2356/3398, 0.5 task/s, elapsed: 5067s, ETA:  2241s2356
[>>>>>>>>>>>>>>>>>>>>          ] 2357/3398, 0.5 task/s, elapsed: 5069s, ETA:  2239s2357
[>>>>>>>>>>>>>>>>>>>>          ] 2358/3398, 0.5 task/s, elapsed: 5071s, ETA:  2237s2358
[>>>>>>>>>>>>>>>>>>>>          ] 2359/3398, 0.5 task/s, elapsed: 5074s, ETA:  2235s2359
[>>>>>>>>>>>>>>>>>>>>          ] 2360/3398, 0.5 task/s, elapsed: 5076s, ETA:  2233s2360
[>>>>>>>>>>>>>>>>>>>>          ] 2361/3398, 0.5 task/s, elapsed: 5079s, ETA:  2231s2361
[>>>>>>>>>>>>>>>>>>>>          ] 2362/3398, 0.5 task/s, elapsed: 5081s, ETA:  2228s2362
[>>>>>>>>>>>>>>>>>>>>          ] 2363/3398, 0.5 task/s, elapsed: 5083s, ETA:  2226s2363
[>>>>>>>>>>>>>>>>>>>>          ]

[>>>>>>>>>>>>>>>>>>>>>         ] 2446/3398, 0.5 task/s, elapsed: 5264s, ETA:  2049s2446
[>>>>>>>>>>>>>>>>>>>>>         ] 2447/3398, 0.5 task/s, elapsed: 5266s, ETA:  2047s2447
[>>>>>>>>>>>>>>>>>>>>>         ] 2448/3398, 0.5 task/s, elapsed: 5269s, ETA:  2045s2448
[>>>>>>>>>>>>>>>>>>>>>         ] 2449/3398, 0.5 task/s, elapsed: 5271s, ETA:  2043s2449
[>>>>>>>>>>>>>>>>>>>>>         ] 2450/3398, 0.5 task/s, elapsed: 5274s, ETA:  2041s2450
[>>>>>>>>>>>>>>>>>>>>>         ] 2451/3398, 0.5 task/s, elapsed: 5276s, ETA:  2038s2451
[>>>>>>>>>>>>>>>>>>>>>         ] 2452/3398, 0.5 task/s, elapsed: 5278s, ETA:  2036s2452
[>>>>>>>>>>>>>>>>>>>>>         ] 2453/3398, 0.5 task/s, elapsed: 5280s, ETA:  2034s2453
[>>>>>>>>>>>>>>>>>>>>>         ] 2454/3398, 0.5 task/s, elapsed: 5283s, ETA:  2032s2454
[>>>>>>>>>>>>>>>>>>>>>         ] 2455/3398, 0.5 task/s, elapsed: 5285s, ETA:  2030s2455
[>>>>>>>>>>>>>>>>>>>>>         ] 2456/3398, 0.5 task/s, elapsed: 5287s, ETA:  2028s2456
[>>>>>>>>>>>>>>>>>>>>>         ]

[>>>>>>>>>>>>>>>>>>>>>>        ] 2539/3398, 0.5 task/s, elapsed: 5465s, ETA:  1849s2539
[>>>>>>>>>>>>>>>>>>>>>>        ] 2540/3398, 0.5 task/s, elapsed: 5468s, ETA:  1847s2540
[>>>>>>>>>>>>>>>>>>>>>>        ] 2541/3398, 0.5 task/s, elapsed: 5470s, ETA:  1845s2541
[>>>>>>>>>>>>>>>>>>>>>>        ] 2542/3398, 0.5 task/s, elapsed: 5473s, ETA:  1843s2542
[>>>>>>>>>>>>>>>>>>>>>>        ] 2543/3398, 0.5 task/s, elapsed: 5475s, ETA:  1841s2543
[>>>>>>>>>>>>>>>>>>>>>>        ] 2544/3398, 0.5 task/s, elapsed: 5477s, ETA:  1838s2544
[>>>>>>>>>>>>>>>>>>>>>>        ] 2545/3398, 0.5 task/s, elapsed: 5479s, ETA:  1836s2545
[>>>>>>>>>>>>>>>>>>>>>>        ] 2546/3398, 0.5 task/s, elapsed: 5481s, ETA:  1834s2546
[>>>>>>>>>>>>>>>>>>>>>>        ] 2547/3398, 0.5 task/s, elapsed: 5483s, ETA:  1832s2547
[>>>>>>>>>>>>>>>>>>>>>>        ] 2548/3398, 0.5 task/s, elapsed: 5485s, ETA:  1830s2548
[>>>>>>>>>>>>>>>>>>>>>>        ] 2549/3398, 0.5 task/s, elapsed: 5488s, ETA:  1828s2549
[>>>>>>>>>>>>>>>>>>>>>>        ]

[>>>>>>>>>>>>>>>>>>>>>>>       ] 2632/3398, 0.5 task/s, elapsed: 5665s, ETA:  1649s2632
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2633/3398, 0.5 task/s, elapsed: 5667s, ETA:  1647s2633
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2634/3398, 0.5 task/s, elapsed: 5670s, ETA:  1644s2634
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2635/3398, 0.5 task/s, elapsed: 5672s, ETA:  1642s2635
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2636/3398, 0.5 task/s, elapsed: 5674s, ETA:  1640s2636
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2637/3398, 0.5 task/s, elapsed: 5675s, ETA:  1638s2637
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2638/3398, 0.5 task/s, elapsed: 5677s, ETA:  1636s2638
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2639/3398, 0.5 task/s, elapsed: 5679s, ETA:  1633s2639
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2640/3398, 0.5 task/s, elapsed: 5681s, ETA:  1631s2640
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2641/3398, 0.5 task/s, elapsed: 5684s, ETA:  1629s2641
[>>>>>>>>>>>>>>>>>>>>>>>       ] 2642/3398, 0.5 task/s, elapsed: 5686s, ETA:  1627s2642
[>>>>>>>>>>>>>>>>>>>>>>>       ]

In [ ]:
sub_dict = {}
sub_dict['results'] = []
for result,img_name in zip(outputs,file_names ):
    img_path = image_dir+img_name
    image = Image.open(img_path)
    (im_width, im_height) = image.size
    image_np=load_image_into_numpy_array(image)
    boxes,labels,scores = mmresult_to_boxes_labels(result)
    #overlay_boxes(image_np, boxes,labels)
    
    #print (boxes)
    image_dict = {}
    image_dict['filename'] = img_name
    image_dict['rects'] = []
    for box, score, label in zip(boxes, scores, labels):
        # scores are sorted so we can break
        print (box, score, label)
        if score < 0.5:
            break
        box_dict = {}
        color = 'red' #label_color(label)
        box_dict['xmin'] = int(box[0])
        box_dict['ymin'] = int(box[1])
        box_dict['xmax'] = int(box[2])
        box_dict['ymax'] = int(box[3])
        box_dict['label'] = int(label)
        box_dict["confidence"] = float(score)
        image_dict['rects'].append(box_dict)
            
    sub_dict['results'].append(image_dict)
    
with open("record_b.json","w") as f:
    json.dump(sub_dict,f)
    print("加载入文件完成...")
    

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
color_dict = {1:(255,0,0),2:(0,255,0),3:(0,0,255),4:(255,255,0),5:(0,255,255)}
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def overlay_boxes(image, boxes,labels):
    """
    Adds the predicted boxes on top of the image

    Arguments:
        image (np.ndarray): an image as returned by OpenCV
        predictions (BoxList): the result of the computation by the model.
            It should contain the field `labels`.
    """
#     labels = predictions.get_field("labels")
#     boxes = predictions.bbox

    colors = [color_dict[int(label)] for label in labels]

    for box, color in zip(boxes, colors):
        box = [int(i) for i in box]
        top_left, bottom_right = box[:2], box[2:]
        cv2.rectangle(
            image, tuple(top_left), tuple(bottom_right), color, 2
        )
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.show()
    return image

def mmresult_to_boxes_labels(result):
    boxes = []
    labels = []
    scores = []
    for i in range(5):
        for one in result[i]:
            tmp = list(one)
            if tmp[4]>0.60:
                boxes.append(tmp[:4])
                scores.append(tmp[4])
                labels.append(i+1)
    return boxes,labels,scores
    


In [ ]:
file_names

In [ ]:
len(sub_dict['results'])